In [27]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif_data(image_path):
    """
    Récupère les données EXIF de l'image.
    """
    image = Image.open(image_path)
    exif_data = image._getexif()
    if exif_data:
        return {TAGS.get(tag, tag): value for tag, value in exif_data.items()}
    return None

def get_geotagging(exif):
    """
    Extrait les informations GPS des données EXIF.
    """
    if 'GPSInfo' in exif:
        gps_info = exif['GPSInfo']
        geotags = {GPSTAGS.get(t, t): gps_info[t] for t in gps_info}
        return geotags
    return None

def get_decimal_coordinates(geotags):
    """
    Convertit les coordonnées GPS EXIF en décimal.
    """
    def convert_to_degrees(value):
        d, m, s = value
        return d + (m / 60.0) + (s / 3600.0)

    lat = convert_to_degrees(geotags['GPSLatitude'])
    lon = convert_to_degrees(geotags['GPSLongitude'])

    if geotags['GPSLatitudeRef'] != "N":
        lat = -lat
    if geotags['GPSLongitudeRef'] != "E":
        lon = -lon

    return lat, lon

# Exemple d'utilisation
image_path = "photo.jpg"  # Chemin vers la photo
exif_data = get_exif_data(image_path)
if exif_data:
    geotags = get_geotagging(exif_data)
    if geotags:
        latitude, longitude = get_decimal_coordinates(geotags)
        print(f"Coordonnées GPS : Latitude={latitude}, Longitude={longitude}")
    else:
        print("Aucune information GPS trouvée dans l'image.")
else:
    print("Aucune donnée EXIF trouvée.")

Coordonnées GPS : Latitude=48.87432098361111, Longitude=2.3388557433333337


In [13]:
import requests

def get_places_osm(lat, lon, radius=1000):
    """
    Requête Overpass API pour obtenir des lieux autour d'une position GPS.
    """
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (
      node["tourism"](around:{radius},{lat},{lon});
      node["amenity"](around:{radius},{lat},{lon});
      node["leisure"](around:{radius},{lat},{lon});
    );
    out center;
    """
    response = requests.get(overpass_url, params={'data': query})
    if response.status_code == 200:
        data = response.json()
        places = []
        for element in data['elements']:
            if 'tags' in element and 'name' in element['tags']:
                places.append({
                    "name": element['tags']['name'],
                    "category": element['tags'].get("tourism", element['tags'].get("amenity", "unknown")),
                })
        return places
    else:
        raise Exception(f"Erreur avec Overpass API : {response.status_code}")

# Utilisation
#latitude, longitude = 48.8588443, 2.2943506  # Coordonnées GPS (exemple : Tour Eiffel)
places = get_places_osm(latitude, longitude, radius=50)
for place in places:
    print(f"{place['name']} (Catégorie : {place['category']})")


Mûre (Catégorie : restaurant)
Long & Tee (Catégorie : cafe)
Mian Fan (Catégorie : restaurant)
Mian Fan (Catégorie : restaurant)


In [29]:
def get_flickr_photos(lat, lon, api_key):
    """
    Requête Flickr API pour obtenir des photos basées sur les coordonnées GPS.
    """
    flickr_url = "https://www.flickr.com/services/rest/"
    params = {
        "method": "flickr.photos.search",
        "api_key": api_key,
        "lat": lat,
        "lon": lon,
        "format": "json",
        "nojsoncallback": 1,
        "radius": 0.02,  # Rayon en km
    }
    response = requests.get(flickr_url, params=params)
    if response.status_code == 200:
        data = response.json()
        photos = []
        for photo in data["photos"]["photo"]:
            photo_url = f"https://live.staticflickr.com/{photo['server']}/{photo['id']}_{photo['secret']}.jpg"
            photos.append(photo_url)
        return photos
    else:
        raise Exception(f"Erreur avec Flickr API : {response.status_code}")

# Exemple d'utilisation
flickr_api_key = "0bb9589f8a2a38cc2f51026e75e4e37f"
photos = get_flickr_photos(latitude, longitude, flickr_api_key)
for photo in photos:
    print(photo)


https://live.staticflickr.com/792/41385154891_8c748a2c76.jpg
https://live.staticflickr.com/4724/25286773007_5ee8da4396.jpg
https://live.staticflickr.com/8003/7625094868_7571cca18b.jpg
https://live.staticflickr.com/7264/7153465613_97841f00c9.jpg
https://live.staticflickr.com/2477/5819178085_97b21e6dd1.jpg
https://live.staticflickr.com/2500/4110873250_f98b216f62.jpg
https://live.staticflickr.com/3030/2999674047_bb4787d76c.jpg
